In [1]:
import torch
import torchaudio
import numpy as np
from types import SimpleNamespace
from sklearn.metrics import average_precision_score
from datasets import load_dataset, Audio
from timm.optim import Mars
from fastprogress import master_bar, progress_bar
from torchvision.transforms.v2 import RandomCrop, CenterCrop

In [ ]:
torch.set_float32_matmul_precision('high')
device="cuda:1"
audioset = load_dataset("danjacobellis/audioset_opus_24kbps",split='train').cast_column('opus', Audio(decode=False))
dataset_train = audioset.select(range(1800000))
dataset_valid = audioset.select(range(1800000,audioset.num_rows))

In [ ]:
config = SimpleNamespace()
# Training and optimizer config
config.batch_size = 128
config.steps_per_epoch = dataset_train.num_rows//config.batch_size
config.grad_accum_steps = 1
config.max_lr = (config.batch_size/128)*6e-4
config.min_lr = config.max_lr/100
config.plot_update = 128
config.epochs = 200
config.lr_scale = 0.5
config.lr_offset = 0.25
config.lr_pow = 2
config.weight_decay = 0.
config.num_workers = 24
config.audio_len = 483840
config.crop_size = 512*512

# model config
config.channels = 1
config.J = 10
config.embed_dim = 256
config.dim_head = 32
config.exp_ratio = 4.0
config.classifier_num_classes = 632

In [ ]:
import torch
import einops
from timm.models.maxxvit import LayerScale

class RMSNormAct(torch.nn.Module):
    def __init__(self, normalized_features):
        super(RMSNormAct, self).__init__()
        self.norm = torch.nn.RMSNorm(normalized_features)
        self.act = torch.nn.GELU()

    def forward(self, x):
        x = self.norm(x)
        x = self.act(x)
        return x

class InvertedResidual1D(torch.nn.Module):
    def __init__(self, in_dim, out_dim, sequence_dim, exp_ratio):
        super(InvertedResidual1D, self).__init__()
        self.exp_dim = int(in_dim * exp_ratio)
        self.pw_exp = torch.nn.Sequential(
            torch.nn.Conv1d(in_dim, self.exp_dim, kernel_size=1, stride=1, bias=False),
            RMSNormAct((self.exp_dim, sequence_dim))
        )
        self.dw_mid = torch.nn.Sequential(
            torch.nn.Conv1d(self.exp_dim, self.exp_dim, kernel_size=3, stride=1, padding=1, groups=self.exp_dim, bias=False),
            RMSNormAct((self.exp_dim, sequence_dim))
        )
        self.se = torch.nn.Identity()
        self.pw_proj = torch.nn.Sequential(
            torch.nn.Conv1d(self.exp_dim, out_dim, kernel_size=1, stride=1, bias=False),
            torch.nn.RMSNorm((out_dim, sequence_dim)) 
        )
        self.dw_end = torch.nn.Identity()
        self.layer_scale = LayerScale(out_dim)
        self.drop_path = torch.nn.Identity()
        
    def forward(self, x):
        shortcut = x if x.shape[1] == self.pw_proj[0].out_channels else None
        x = self.pw_exp(x)
        x = self.dw_mid(x)
        x = self.se(x)
        x = self.pw_proj(x)
        x = self.dw_end(x)
        x = self.layer_scale(x)
        x = self.drop_path(x)
        if shortcut is not None:
            x += shortcut
        return x

class TransformerBlock1D(torch.nn.Module):
    def __init__(self, embed_dim, dim_feedforward, nhead):
        super().__init__()
        self.layer = torch.nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=0.0,
            activation='gelu',
            batch_first=True
        )
    def forward(self, x):
        x = einops.rearrange(x, 'b c s -> b s c')
        x = self.layer(x)
        x = einops.rearrange(x, 'b s c -> b c s')
        return x

class AsCAN1D(torch.nn.Module):
    def __init__(self, input_dim, embed_dim, sequence_dim, dim_head, exp_ratio):
        super().__init__()
        C=lambda:InvertedResidual1D(embed_dim, embed_dim, sequence_dim, exp_ratio)
        T=lambda:TransformerBlock1D(embed_dim, int(exp_ratio*embed_dim), embed_dim//dim_head)
        self.layers=torch.nn.Sequential(
            torch.nn.Conv1d(input_dim,embed_dim,kernel_size=1),
            RMSNormAct((embed_dim, sequence_dim)),
            C(),C(),C(),T(),
            C(),C(),T(),T(),
            C(),T(),T(),T()
        )
    def forward(self,x):
        return self.layers(x)

class WaveletPooling1D(torch.nn.Module):
    def __init__(self, embed_dim, sequence_dim, wpt, num_levels):
        super().__init__()
        self.wpt = wpt
        self.num_levels = num_levels
        current_sequence_dim = sequence_dim
        self.projection_down = torch.nn.ModuleList()
        for _ in range(num_levels):
            self.projection_down.append(
                torch.nn.Sequential(
                    torch.nn.Conv1d(embed_dim, embed_dim // 2, kernel_size=1, padding=0),
                    torch.nn.RMSNorm((embed_dim // 2, current_sequence_dim))
                )
            )
            current_sequence_dim //= 2
    def forward(self, x):
        for i in range(self.num_levels):
            x = self.projection_down[i](x)
            x = self.wpt.analysis_one_level(x)
        return x


class TFTClassifier1D(torch.nn.Module):
    def __init__(self, config, wpt):
        super().__init__()
        self.wpt = wpt
        self.ascan = AsCAN1D(
            input_dim=config.channels*(2**config.J),
            embed_dim=config.embed_dim,
            sequence_dim=config.crop_size//(2**config.J),
            dim_head=config.dim_head,
            exp_ratio=config.exp_ratio
        )
        self.pool = WaveletPooling1D(
            embed_dim=config.embed_dim,
            sequence_dim=config.crop_size//(2**config.J),
            wpt=wpt,
            num_levels=int(np.log2(config.crop_size) - config.J)
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Conv1d(config.embed_dim, config.classifier_num_classes, kernel_size=1),
            torch.nn.Flatten()
        )
    def forward(self,x):
        x = self.wpt(x)
        x = self.ascan(x)
        x = self.pool(x)
        return self.classifier(x)

In [ ]:
from pytorch_wavelets import DWT1DForward
from tft.transforms import WPT1D

wt = DWT1DForward(J=1, mode='periodization', wave='bior4.4')
wpt = WPT1D(wt,J=config.J).to(device)

model = TFTClassifier1D(config,wpt).to(device)

for name, module in model.named_children():
    print(f"{sum(p.numel() for p in module.parameters())/1e6} \t {name}")

In [ ]:
rand_crop = RandomCrop(
    size=(1,config.crop_size),
    pad_if_needed=True,
)

cent_crop = CenterCrop(
    size=(1,config.crop_size),
)

def train_collate_fn(batch):
    B = len(batch)
    x = torch.zeros((B, config.channels, 1, config.crop_size), dtype=torch.float)
    y = []
    for i_sample, sample in enumerate(batch):
        y.append(sample['label'])
        x_raw, fs = torchaudio.load(uri = sample['opus']['bytes'],normalize=False)
        x[i_sample,:,:,:] = rand_crop(x_raw.unsqueeze(0).unsqueeze(0))
    return x[:,:,0,:], y

def valid_collate_fn(batch):
    B = len(batch)
    x = torch.zeros((B, config.channels, 1, config.crop_size), dtype=torch.float)
    y = []
    for i_sample, sample in enumerate(batch):
        y.append(sample['label'])
        x_raw, fs = torchaudio.load(uri = sample['opus']['bytes'],normalize=False)
        x[i_sample,:,:,:] = cent_crop(x_raw.unsqueeze(0).unsqueeze(0))
    return x[:,:,0,:], y

def create_multi_hot_labels(batch_of_label_indices, num_classes):
    batch_size = len(batch_of_label_indices)
    labels = torch.zeros(batch_size, num_classes, dtype=torch.float32)
    for i, sample_labels in enumerate(batch_of_label_indices):
        indices = torch.tensor(sample_labels, dtype=torch.long)
        labels[i].scatter_(0, indices, 1.0)
    return labels

In [ ]:
optimizer = Mars(
    params=model.parameters(),
    lr=config.min_lr,
    weight_decay=config.weight_decay,
    caution=True
)

def EmPL(x):
    return np.exp(-np.power(np.log(x), config.lr_pow))
        
def EmPL_sched(i_step, config):
    x = i_step / (config.steps_per_epoch/config.plot_update)
    sched = EmPL(x/(config.lr_scale*config.steps_per_epoch/config.plot_update)+config.lr_offset)
    scale = config.max_lr
    lr = scale * sched
    return lr / config.min_lr
    
schedule = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda = lambda i_step: EmPL_sched(i_step, config),
)

In [ ]:
# learning_rates = [optimizer.param_groups[0]['lr']]
# i_step = 0
# for i_epoch in range(config.epochs):
#     for i_batch in range(config.steps_per_epoch):
#         if (i_step+1) % config.plot_update == 0:
#             schedule.step()
#             learning_rates.append(optimizer.param_groups[0]['lr'])
#         i_step+=1
# import matplotlib.pyplot as plt
# plt.plot(learning_rates)

In [ ]:
mb = master_bar(range(config.epochs))
mb.names = ['per batch','smoothed']
train_loss = []
mAP = []
learning_rates = [optimizer.param_groups[0]['lr']]
i_step = 0
for i_epoch in mb:
    # training
    model.train()
    dataloader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=train_collate_fn
    )
    pb = progress_bar(dataloader_train, parent=mb)
    for i_batch, (x,y) in enumerate(pb):
        x = x.to(device)
        y = create_multi_hot_labels(y, config.classifier_num_classes).to(device)
        logits = model(x)
        loss = torch.nn.BCEWithLogitsLoss()(logits, y)

        train_loss.append(loss.item())
        loss.backward()
        if (i_batch + 1) % config.grad_accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # plotting and learning rate update
        if (i_step+1) % config.plot_update == 0:
            plot_n = len(train_loss) // config.plot_update
            smoothed_x = (0.5+torch.arange(plot_n)) * config.plot_update
            smoothed_y = torch.tensor(train_loss).reshape(plot_n, -1).mean(dim=1)
            train_x = range(len(train_loss))
            train_y = train_loss
            mb.update_graph([[train_x, np.log10(train_y)],[smoothed_x, np.log10(smoothed_y)]])

            # lr update
            schedule.step()
            learning_rates.append(optimizer.param_groups[0]['lr'])
    
        i_step+=1
        
    model.eval()
    all_logits = []
    all_labels = []
    data_loader_valid = torch.utils.data.DataLoader(
        dataset_valid,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        drop_last=False,
        pin_memory=True,
        collate_fn=valid_collate_fn
    )
    pb_valid = progress_bar(data_loader_valid, parent=mb)
    for i_batch, (x, y) in enumerate(pb_valid):
        x = x.to(device)
        y = create_multi_hot_labels(y, config.classifier_num_classes).to(device)
        with torch.no_grad():
            logits = model(x)
        all_logits.append(logits.cpu())
        all_labels.append(y.cpu())
    all_logits = torch.cat(all_logits, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    all_probs = all_logits.sigmoid().numpy()
    all_labels_np = all_labels.numpy()    
    ap_per_class = []
    num_classes = all_probs.shape[1]
    labels_per_class = np.sum(all_labels_np, axis=0)
    for class_idx in range(num_classes):
        if labels_per_class[class_idx] == 0:
            ap = 0.0
        else:
            ap = average_precision_score(all_labels_np[:, class_idx],
                                         all_probs[:, class_idx])
        ap_per_class.append(ap)
    mAP_value = np.mean(ap_per_class)
    mAP.append(mAP_value)
    mb.main_bar.comment = f'mAP {mAP_value:.4g}'

    torch.save({
        'i_epoch': i_epoch,
        'learning_rates': learning_rates,
        'smoothed_y': smoothed_y,
        'mAP': mAP,
        'config': config,
        'model_state_dict': model.state_dict(),
    }, f"log_{device}.pth")